In [160]:
from sys import path
path.append('../../src/')
path.append('../rcwa_tf/src/')
import tensorflow as tf
import solver
import solver_metasurface
import numpy as np
import importlib
importlib.reload(solver_metasurface)
importlib.reload(solver)
from copy import deepcopy
p = {}
p['enable_print'] = True
p['pixelsX'] = 100
p['pixelsY'] = 1
p['N'] = 50
p['sigmoid_update'] = 40
p['learning_rate'] = 0.01
p['parameter_string'] = 'N' + str(p['N']) \
    + '-sigmoid_update' + str(p['sigmoid_update']) \
    + '-learning_rate' + str(p['learning_rate'])
step = 30
wavelengths = np.arange(120, 320 + step, step = step)
passthrough_band_indices = [5,6]
determinator = np.array([-1/(len(wavelengths) - len(passthrough_band_indices))] * len(wavelengths))
determinator[passthrough_band_indices] = 1/len(passthrough_band_indices)
# determinator = tf.constant([[determinator[i]] * 200 for i in range(len(determinator))]) # Find out where this comes from
p['thetas'] = np.zeros(len(wavelengths), dtype = 'float32')
p['phis'] = deepcopy(p['thetas'])
p['pte'] = deepcopy(p['thetas']) + 1
p['ptm'] = deepcopy(p['thetas'])
p['Lx'] = p['Ly'] = 100 # Nanometers
p['erd'] = p['ers'] = 3.4
p['L'] = np.zeros(shape = (8,)) + 50.0 #THIS IS THE NUMBER OF LAYERS, WTF IS GOING ON????????
# p['L'] = deepcopy(p['thetas']) + 50.0
p['f'] = 0.0
p['initial_k'] = np.zeros(shape = (len(p['L']) - 1, p['pixelsX'], p['pixelsY'])) + 3.4
for i in range(len(p['initial_k'])):
    for j in range(len(p['initial_k'][i])):
        for k in range(len(p['initial_k'][i][j])):
            if (i+j+k)%2:
                p['initial_k'][i][j][k] += 3.4
p['PQ'] = [3,3]
PQ = p['PQ']
p['upsample'] = 2 # Density of measurements
p['sigmoid_coeff'] = 0.1
p['enable_random_init'] = False
p['enable_debug'] = False
p['enable_print'] = True
p['enable_timing'] = True
p['enable_logging'] = True
rx = int(p['Lx'] * p['pixelsX'])
ry = int(p['Ly'] * p['pixelsY'])
p['wavelengths'] = [120.0]
PQ_f = tf.convert_to_tensor(p['PQ'], dtype=tf.float32)

In [161]:
def init_scaled_metasurface(params):
    if params['enable_random_init']:
        return np.random.rand(params['Nlay'] - 1, params['pixelsX'], params['pixelsY']) * params['ers']
    else:
        return params['initial_k']

In [162]:
params = solver.initialize_params(wavelengths=p['wavelengths'],
                                      thetas=p['thetas'],
                                      phis=p['phis'],
                                      pte=p['pte'],
                                      ptm=p['ptm'],
                                      pixelsX=p['pixelsX'],
                                      pixelsY=p['pixelsY'],
                                      erd=p['erd'],
                                      ers=p['ers'],
                                      PQ=p['PQ'],
                                      Lx=p['Lx'],
                                      Ly=p['Ly'],
                                      L=p['L'],
                                      Nx=16,
                                      eps_min=0.0,
                                      eps_max=p['erd'])
    
# Merge with the user-provided parameter dictionary.
params['N'] = p['N']
params['sigmoid_coeff'] = p['sigmoid_coeff']
params['sigmoid_update'] = p['sigmoid_update']
params['learning_rate'] = p['learning_rate']
# params['focal_spot_radius'] = p['focal_spot_radius']
params['enable_random_init'] = p['enable_random_init']
# params['initial_height'] = p['initial_height']
params['enable_debug'] = p['enable_debug']
params['enable_print'] = p['enable_print']
params['enable_logging'] = p['enable_logging']
params['initial_k'] = p['initial_k']

# Get the loss function.
# loss_function = p['loss_function']

# This flag is set if the solver encounters an error.
params['err'] = False

# Define the free-space propagator and input field distribution
# for the metasurface.
params['f'] = p['f'] * 1E-9
params['upsample'] = p['upsample']
params['propagator'] = solver.make_propagator(params, params['f'])
params['input'] = solver.define_input_fields(params)
# Change to scaled metasurface
# TODO change
k = tf.Variable(
    init_scaled_metasurface(params),#, initial_height=params['initial_height']),
    dtype=tf.float32)
print(k.shape)

(7, 100, 1)


In [163]:
print(np.sum(k ** 2))
print(k.shape)

20230.0
(7, 100, 1)


In [164]:
print(k)

<tf.Variable 'Variable:0' shape=(7, 100, 1) dtype=float32, numpy=
array([[[3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
        [3.4],
        [6.8],
    

In [165]:
ER_t, UR_t = solver_metasurface.generate_scaled_metasurface(k, params)

8091.9995
(7, 100, 1)
(1, 100, 1, 7, 16, 16)


In [166]:
print(ER_t.shape)
print(np.sum(ER_t ** 2))

(1, 100, 1, 8, 16, 16)
(2367487.8+0j)


In [167]:
import rcwa_utils
ERC = rcwa_utils.convmat(ER_t, PQ[0], PQ[1])
tf.linalg.inv(ERC)

<tf.Tensor: shape=(1, 100, 1, 8, 9, 9), dtype=complex64, numpy=
array([[[[[[0.29411766+0.j, 0.        +0.j, 0.        +0.j, ...,
            0.        +0.j, 0.        +0.j, 0.        +0.j],
           [0.        +0.j, 0.29411766+0.j, 0.        +0.j, ...,
            0.        +0.j, 0.        +0.j, 0.        +0.j],
           [0.        +0.j, 0.        +0.j, 0.29411766+0.j, ...,
            0.        +0.j, 0.        +0.j, 0.        +0.j],
           ...,
           [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
            0.29411766+0.j, 0.        +0.j, 0.        +0.j],
           [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
            0.        +0.j, 0.29411766+0.j, 0.        +0.j],
           [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
            0.        +0.j, 0.        +0.j, 0.29411766+0.j]],

          [[0.29411766+0.j, 0.        +0.j, 0.        +0.j, ...,
            0.        +0.j, 0.        +0.j, 0.        +0.j],
           [0.        +0.j, 0.294117

In [168]:
ER_t

<tf.Tensor: shape=(1, 100, 1, 8, 16, 16), dtype=complex64, numpy=
array([[[[[[3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           ...,
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j]],

          [[3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           ...,
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j,
            3.4+0.j],
           [3.4+0.j, 3.4+0.j, 3.4+0.j, ..., 3.4+0.j, 3.4+0.j

In [169]:
print(np.mean(ER_t, axis = (-1, -2, -3)))
print(np.mean(UR_t, axis = (-1, -2, -3)))

[[[3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.4000006+0.j]
  [3.40000